# Virtual screening

Ligand-based virtual screening (LBVS) involves predicting bioactivity of large collections of compounds. It involves multiple steps, which are quite easy to perform with scikit-fingerprints.

Notebook inspired by [Tutorial for the Teach-Discover-Treat (TDT) Competition 2014](https://github.com/sriniker/TDT-tutorial-2014/tree/master) by Sereina Riniker and Gregory Landrum.

First, we will load the Malaria HTS dataset from the above competition. It was created in a series of high-throughput screening (HTS) campaigns from around 2010-2012. It contains an already binarized bioactivity labels.

For shorter calculations in this tutorial, we will subsample the negative class to 100 thousand samples.

In [1]:
import pandas as pd
import numpy as np


df = pd.read_parquet("data/malaria_hts_train.parquet")

# remove ambiguous labels
df = df[df["label"] != "ambiguous"]

# get binary integer labels
df["label"] = df["label"].map({"false": 0, "true": 1})

smiles_list = df["SMILES"].values
labels = df["label"].values

# subsample
pos_idx = np.where(labels == 1)[0]
neg_idx = np.random.choice(np.where(labels == 0)[0], size=100_000, replace=False)
idx = np.concatenate([pos_idx, neg_idx])
smiles_list = smiles_list[idx].tolist()
labels = labels[idx]

print(f"Positive samples: {len(pos_idx)}")
print(f"Negative samples: {len(neg_idx)}")
print(f"Shapes: SMILES list {len(smiles_list)}, labels {len(labels)}")

Positive samples: 1528
Negative samples: 100000
Shapes: SMILES list 101528, labels 101528


**Exercise 1**

This dataset contains some invalid molecules, e.g. breaking valence rules in modern RDKit. Use `MolFromSmilesTransformer`, and note that:
- some molecules may error out, so use `valid_only` and `.transform_x_y()`
- you can parallelize it with `n_jobs` and `batch_size` for efficiency, just like fingerprints.

Use [documentation](https://scikit-fingerprints.readthedocs.io/latest/api_reference.html) if necessary.

In [2]:
from skfp.preprocessing import MolFromSmilesTransformer

mol_from_smiles = MolFromSmilesTransformer(
    valid_only=True, n_jobs=-1, batch_size=1000, verbose=True
)

mols, labels = mol_from_smiles.transform_x_y(smiles_list, labels)

print(f"Removed molecules: {len(smiles_list) - len(mols)}")

  0%|          | 0/101 [00:00<?, ?it/s]

Removed molecules: 0


## Molecular filters

Typically, in LBVS we have huge collections of molecules. Many of those are highly reactive, promiscuous compounds, dyes, or contain toxic functional groups. Thus, we use **molecular filters** to remove them. They are sets of rules to remove unwanted compounds.

They can be divided into 2 groups:

1. **Physicochemical filters** - define allowed ranges of physicochemical descriptors (e.g. molecular weight, logP, HBA, HBD) and keep only molecules fulfilling those requirements. Examples include [Lipinski's rule of 5](https://scikit-fingerprints.readthedocs.io/latest/modules/generated/skfp.filters.LipinskiFilter.html#skfp.filters.LipinskiFilter) and [REOS](https://scikit-fingerprints.readthedocs.io/latest/modules/generated/skfp.filters.REOSFilter.html#skfp.filters.REOSFilter).

2. **Substructural filters** - define unwanted substructures with SMARTS and keep only molecules that do not contain those patterns. Examples include [PAINS](https://scikit-fingerprints.readthedocs.io/latest/modules/generated/skfp.filters.PAINSFilter.html) and [Brenk filter](https://scikit-fingerprints.readthedocs.io/latest/modules/generated/skfp.filters.BrenkFilter.html).

RDKit contains 10 substructural filters, but no physicochemical filters. Scikit-fingerprints implements 31 filters, 21 physicochemical and 10 substructural. API is based on [feature-engine](https://feature-engine.trainindata.com/en/latest/) and other scikit-learn compatible libraries for preprocessing data.

Let's see an example for a substructural filter with RDKit.

In [3]:
from rdkit.Chem import FilterCatalog
from rdkit.Chem.rdfiltercatalog import FilterCatalogParams
from tqdm.notebook import tqdm

filter_catalog_pains_a = FilterCatalogParams.FilterCatalogs.PAINS_A
params = FilterCatalog.FilterCatalogParams()
params.AddCatalog(filter_catalog_pains_a)
filters = FilterCatalog.FilterCatalog(params)

mols_filtered_rdkit = []
labels_filtered_rdkit = []
for mol, label in tqdm(zip(mols, labels), total=len(mols)):
    matches = filters.GetMatches(mol)
    if not matches:
        mols_filtered_rdkit.append(mol)
        labels_filtered_rdkit.append(label)


<frozen importlib._bootstrap>:488: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterHierarchyMatcher> already registered; second conversion method ignored.
<frozen importlib._bootstrap>:488: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry> already registered; second conversion method ignored.


  0%|          | 0/101528 [00:00<?, ?it/s]

And with scikit-fingerprints:

In [4]:
from skfp.filters import PAINSFilter


filter_pains = PAINSFilter(variant="A", n_jobs=-1, verbose=True)

mols_filtered_skfp, labels_filtered_skfp = filter_pains.transform_x_y(mols, labels)

  0%|          | 0/32 [00:00<?, ?it/s]

<frozen importlib._bootstrap>:488: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterHierarchyMatcher> already registered; second conversion method ignored.
<frozen importlib._bootstrap>:488: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry> already registered; second conversion method ignored.
<frozen importlib._bootstrap>:488: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterHierarchyMatcher> already registered; second conversion method ignored.
<frozen importlib._bootstrap>:488: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry> already registered; second conversion method ignored.
<frozen importlib._bootstrap>:488: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterHierarchyMatcher> already registered; second conversion method ignored.
<frozen importlib._bootstrap>:488: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry

### Exercise 2

Lipinski's rule of 5 is arguably the first and most famous molecular filters. It is a physicochemical filter, where a molecule can violate at most one of the rules:

- molecular weight $\leq 500$
- hydrogen bond acceptors (HBA) $\leq 10$
- hydrogen bond donors (HBD) $\leq 5$
- logP $\leq 5$

Implement the Lipinski's rule using RDKit and `for` loop, and filter molecules. Remember to also filter labels for consistency! Relevant RDKit descriptors are in [rdkit.Chem.Descriptors](https://www.rdkit.org/docs/source/rdkit.Chem.Descriptors.html), [rdkit.Chem.rdMolDescriptors](https://www.rdkit.org/docs/source/rdkit.Chem.rdMolDescriptors.html), and [rdkit.Chem.Crippen](https://www.rdkit.org/docs/source/rdkit.Chem.Crippen.html) modules.

Then, use the [scikit-fingerprints LipinskiFilter](https://scikit-fingerprints.readthedocs.io/latest/modules/generated/skfp.filters.LipinskiFilter.html#skfp.filters.LipinskiFilter) and compare the two.

In [ ]:
from rdkit.Chem.Crippen import MolLogP
from rdkit.Chem.Descriptors import MolWt
from rdkit.Chem.rdMolDescriptors import CalcNumHBA, CalcNumHBD


mols_filtered_rdkit = []
labels_filtered_rdkit = []
for mol, label in tqdm(zip(mols, labels), total=len(mols)):
    mol_weight = MolWt(mol)
    hba = CalcNumHBA(mol)
    hbd = CalcNumHBD(mol)
    logp = MolLogP(mol)

    if mol_weight <= 500 and hba <= 10 and hbd <= 5 and logp <= 5:
        mols_filtered_rdkit.append(mol)
        labels_filtered_rdkit.append(label)


In [ ]:
from skfp.filters import LipinskiFilter

lipinski_filter = LipinskiFilter(n_jobs=-1, verbose=True)
mols_filtered_skfp, labels_filtered_skfp = lipinski_filter.transform_x_y(mols, labels)

**Exercise 3**

Scikit-learn `Pipeline` does not support `.transform_x_y()`, so in order to use multiple filters, we need to apply them one after another. A common example is using multiple PAINS sets A, B, C.

Create a list of filters `PAINSFilter` with different variants A, B and C. Apply filtering on molecules and labels with all three filters one after another.

Save final results in variables `mols_filtered` and `labels_filtered`, which we will use in the rest of the notebook.

In [ ]:
from skfp.filters import PAINSFilter

pains_filters = [
    PAINSFilter(variant="A", n_jobs=-1),
    PAINSFilter(variant="B", n_jobs=-1),
    PAINSFilter(variant="C", n_jobs=-1),
]

mols_filtered = mols
labels_filtered = labels

for pains_filter_variant in pains_filters:
    mols_filtered, labels_filtered = pains_filter_variant.transform_x_y(
        mols_filtered, labels_filtered
    )


Let's check how many molecules we have after filtering:

In [ ]:
n_active = labels_filtered.sum()
n_inactive = len(labels_filtered) - n_active

print(f"Number of molecules: before {len(mols)}, after {len(mols_filtered)}")
print(f"Inactive molecules: {n_inactive}")
print(f"Active molecules: {n_active}")

## Data splits

In previous notebook, we used predefined splits. For new datasets, we need to compute them explicitly. Doing this with RDKit or DeepChem requires quite a lot of boilerplate code. Let's see the scaffold split as the most popular example, computed with RDKit. Here, the test set consists of the least frequent scaffolds.

In [ ]:
from copy import deepcopy
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit import Chem
from collections import defaultdict

data_size = len(molecules)

test_size = int(0.2 * data_size)
train_size = data_size - test_size

scaffold_sets = defaultdict(list)
for idx, mol in enumerate(mols):
    mol = deepcopy(mol)
    Chem.RemoveStereochemistry(mol)
    scaffold = MurckoScaffold.MurckoScaffoldSmiles(mol=mol)
    scaffold_sets[scaffold].append(idx)

# sort ascending by size of scaffold set
scaffold_sets = list(scaffold_sets.values())
scaffold_sets.sort(key=len)

# assign the least frequent sets to test, rest to train
train_idxs: list[int] = []
test_idxs: list[int] = []
for scaffold_set in scaffold_sets:
    if len(test_idxs) < test_size:
        test_idxs.extend(scaffold_set)
    else:
        train_idxs.extend(scaffold_set)

mols_train = [mols[i] for i in train_idxs]
mols_test = [mols[i] for i in test_idxs]

y_train = labels[train_idxs]
y_test = labels[test_idxs]

In [ ]:
len(mols_train), len(mols_test)

**Exercise 4**

Scikit-fingerprints implements [5 data splitting functions](https://scikit-fingerprints.readthedocs.io/latest/modules/model_selection.html), including train-test and train-valid-test splitters.

Using scikit-fingerprints, implement:
- scaffold splitting
- MaxMin (maximum diverisity) splitting

Use 80-20% train-test proportions. Remember to set `random_state=0` for MaxMin for deterministic results.

Use variable names like `mols_train_scaffold`, `mols_test_scaffold`, `mols_train_maxmin` etc.

In [ ]:
from skfp.model_selection import maxmin_train_test_split, scaffold_train_test_split


mols_train_scaffold, mols_test_scaffold, y_train_scaffold, y_test_scaffold = scaffold_train_valid_test_split(
    mols, labels, test_size=0.2
)
mols_train_maxmin, mols_test_maxmin, y_train_maxmin, y_test_maxmin = maxmin_train_test_split(
    mols, labels, test_size=0.2, random_state=0
)

## Similarity searching LBVS

### Similarities & distances

In some cases we might want to compute how similar two molecules are to each other, or how far they are in some euclidean space mapping. To do that we can compute similarity or distance using molecular fingerprints.

scikit-fingerprints delivers both binary and count variants of similarity and distance computation functions.

Lets try to compute similarity between two binary vectors using `tanimoto_binary_similarity`

In [16]:
bit_vec_1 = np.array([1, 1, 0, 0])
bit_vec_2 = np.array([0, 1, 1, 0])

In [17]:
from skfp.distances import tanimoto_binary_similarity

binary_similarity = tanimoto_binary_similarity(bit_vec_1, bit_vec_2)

print(f"similarity of binary vectors: {binary_similarity}")

similarity of binary vectors: 0.3333333333333333


If we use count vectors we need to use `tanimoto_count_similarity`

In [18]:
count_vec_1 = np.array([2, 3, 4, 0])
count_vec_2 = np.array([2, 3, 4, 2])

In [19]:
from skfp.distances import tanimoto_count_similarity

count_similarity = tanimoto_count_similarity(bit_vec_1, bit_vec_2)

print(f"similarity of count vectors: {count_similarity}")

similarity of count vectors: 0.3333333333333333


We can also use bulk similarity computation to speed up very heavy computations between two sets of molecules.

If in one set we have X molecules and Y molecules in the other set, finding distances or similarities between every pair between X and Y can be very slow. scikit-fingerpirnts implementation allows us to compute them efficiently

In [20]:
from skfp.distances.tanimoto import (
    bulk_tanimoto_binary_similarity
)

arr_1 = np.array(
    [
        [1, 1, 1],
        [0, 0, 1],
        [1, 1, 1],
    ]
)

arr_2 = np.array(
    [
        [1, 0, 1],
        [0, 1, 1],
        [1, 1, 1],
    ]
)

sim = bulk_tanimoto_binary_similarity(arr_1, arr_2)
sim

array([[0.66666667, 0.66666667, 1.        ],
       [0.5       , 0.5       , 0.33333333],
       [0.66666667, 0.66666667, 1.        ]])

Sometimes we want to compute similarity between every two molcules in a dataset. We can compute that by passing just one argument.

In [21]:
X = np.array(
    [
        [1, 0, 1],
        [0, 1, 1],
        [1, 1, 1],
    ]
)

sim = bulk_tanimoto_binary_similarity(X)
sim

array([[1.        , 0.33333333, 0.66666667],
       [0.33333333, 1.        , 0.66666667],
       [0.66666667, 0.66666667, 1.        ]])

### Performance

Lets see the time difference between manual computation of distance matrix between all molecules in a small dataset of 300 molecules.

In [22]:
from skfp.fingerprints import ECFPFingerprint

ecfp_fingerprint = ECFPFingerprint(count=True, n_jobs=-1, verbose=True)

fps = ecfp_fingerprint.transform(molecules[:300])

  0%|          | 0/25 [00:00<?, ?it/s]

Manual implementation

In [23]:
%timeit -r 3 -n 10 [tanimoto_count_similarity(fps[i], fps[j]) for i in range(len(fps)) for j in range(len(fps))]

1.73 s ± 11.1 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


Bulk implementation

In [24]:
from skfp.distances.tanimoto import bulk_tanimoto_count_similarity

In [25]:
%timeit -r 3 -n 10 [bulk_tanimoto_count_similarity(fps)]

30.6 ms ± 19.8 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


### Exercise 5

Smilarities and distances have very similar implementation. Now that we know how to use similarities, let's try to train a K nearest neighbors model using binary variant of `ECFPFingerprint`. To do that we'll use a BACE dataset from MoleculeNet benchmark.

- Load SMILES strings and labels from BACE dataset.
- Perform scaffold train-test split with 8:2 ratio
- Build a scikit-learn pipeline using ECFP fingerprint annd KNeighborsClassfier. Remember to pass binary variant of Tanimoto **distance** as `metric` parameter of the KNN model.
- fit the pipeline on training data and make prediction

In [26]:
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier

# Complete needed imports!
from skfp.distances import tanimoto_binary_distance
from skfp.model_selection import scaffold_train_test_split
from skfp.datasets.moleculenet import load_bace

# load bace dataset
smiles, labels = load_bace()

# perform train/test split. (we don't want to use validation set!)
smiles_train, smiles_test, labels_train, labels_test = scaffold_train_test_split(
    smiles, labels, test_size=0.2
)

# initialize a pipeline consisting of binary ECFP fingerprint
# and KNeighborsClassifier with correct Tanimoto distance metric
pipeline = make_pipeline(
    ECFPFingerprint(n_jobs=-1),
    KNeighborsClassifier(metric=tanimoto_binary_distance, n_jobs=-1),
)

# rain and predict the pipeline
pipeline.fit(smiles_train, labels_train)
labels_pred = pipeline.predict_proba(smiles_test)[:,1]

# Compute auroc metric
auroc_score = roc_auc_score(labels_test, labels_pred)

print(f"AUROC score: {auroc_score}")

AUROC score: 0.7775590551181102


In [27]:
auroc_score

0.7775590551181102

### Multioutput evaluation metrics

Sometimes, the dataset that we use will have multiple labels.

In these scenarios we want to compute aggregated metrics from each predicted class.

To allow that, scikit-fingerprints exports multioutput scoring metrics.

### Exercise 6

- Load sider dataset from MoleculeNet benchmark as SMILES strings and labeks.
- Perform scaffold train-test split with 8:2 ratio.
- Create and fit the pipeline consisting of **count** variant of ECFP fingerprint, and RandomForestClassifier, similarly to the previous exercise.
- Make a probability prediction using `.predict_proba()` method
- Predicted probabilities are stored in a 3-dimensional array. `extract_pos_proba()` function from `metrics` submodule of `skfp` allows to extract relevant probabilities easily.
- Pass correct labels alongside the extracted probabilities to `multioutput_auprc_score` from `metrics` submodule of `skfp`.

In [28]:
from sklearn.ensemble import RandomForestClassifier

# complete imports
from skfp.datasets.moleculenet import load_sider
from skfp.metrics import multioutput_auprc_score, extract_pos_proba

smiles, labels = load_sider()

smiles_train, smiles_test, labels_train, labels_test = scaffold_train_test_split(
    smiles, labels, test_size=0.2
)

pipeline = make_pipeline(ECFPFingerprint(n_jobs=-1, count=True), RandomForestClassifier(n_jobs=-1))

pipeline.fit(smiles_train, labels_train)

labels_proba = pipeline.predict_proba(smiles_test)
labels_proba = extract_pos_proba(labels_proba)

auprc_score = multioutput_auprc_score(labels_test, labels_proba)

print(f"AUPRC score: {auprc_score}")

[13:31:54] WARNING: not removing hydrogen atom without neighbors
[13:31:54] WARNING: not removing hydrogen atom without neighbors
[13:31:54] WARNING: not removing hydrogen atom without neighbors
[13:31:54] WARNING: not removing hydrogen atom without neighbors
[13:31:55] WARNING: not removing hydrogen atom without neighbors
[13:31:55] WARNING: not removing hydrogen atom without neighbors
[13:31:55] WARNING: not removing hydrogen atom without neighbors
[13:31:55] WARNING: not removing hydrogen atom without neighbors
[13:31:55] WARNING: not removing hydrogen atom without neighbors
[13:31:55] WARNING: not removing hydrogen atom without neighbors
[13:31:55] WARNING: not removing hydrogen atom without neighbors
[13:31:55] WARNING: not removing hydrogen atom without neighbors
[13:31:55] WARNING: not removing hydrogen atom without neighbors


AUPRC score: 0.6865983292077823


In [29]:
auprc_score

0.6865983292077823

---